# In Depth - Decision Trees and Forests

In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

Here we'll explore a class of algorithms based on decision trees.
Decision trees at their root are extremely intuitive.  They
encode a series of "if" and "else" choices, similar to how a person might make a decision.
However, which questions to ask, and how to proceed for each answer is entirely learned from the data.

For example, if you wanted to create a guide to identifying an animal found in nature, you
might ask the following series of questions:

- Is the animal bigger or smaller than a meter long?
    + *bigger*: does the animal have horns?
        - *yes*: are the horns longer than ten centimeters?
        - *no*: is the animal wearing a collar
    + *smaller*: does the animal have two or four legs?
        - *two*: does the animal have wings?
        - *four*: does the animal have a bushy tail?

and so on.  This binary splitting of questions is the essence of a decision tree.

One of the main benefit of tree-based models is that they require little preprocessing of the data.
They can work with variables of different types (continuous and discrete) and are invariant to scaling of the features.

Another benefit is that tree-based models are what is called "nonparametric", which means they don't have a fix set of parameters to learn. Instead, a tree model can become more and more flexible, if given more data.
In other words, the number of free parameters grows with the number of samples and is not fixed, as for example in linear models.


## Decision Tree Classification

Decision tree classification work very similarly, by assigning all points within a leaf the majority class in that leaf:

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from figures import plot_2d_separator
from figures import cm2


X, y = make_blobs(centers=[[0, 0], [1, 1]], random_state=61526, n_samples=100)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

clf = DecisionTreeClassifier(max_depth=10)
clf.fit(X_train, y_train)
pred = clf.predict(X_test)

In [ ]:
pred

We can plot the decision boundaries found using the training data.

In [ ]:
plot_2d_separator(clf, X, fill=True)
plt.scatter(X_train[:, 0], X_train[:, 1], c=y_train, s=60, alpha=.7, edgecolor='k');

Similarly, we get the following classification on the testing set.

In [ ]:
plot_2d_separator(clf, X, fill=True)
plt.scatter(X_test[:, 0], X_test[:, 1], c=y_test, cmap=cm2, s=60, edgecolor='k');

There are many parameter that control the complexity of a tree, but the one that might be easiest to understand is the maximum depth. This limits how finely the tree can partition the input space, or how many "if-else" questions can be asked before deciding which class a sample lies in.

This parameter is important to tune for trees and tree-based models. The interactive plot below shows how underfit and overfit looks like for this model. Having a ``max_depth`` of 1 is clearly an underfit model, while a depth of 7 or 8 clearly overfits. The maximum depth a tree can be grown at for this dataset is 8, at which point each leave only contains samples from a single class. This is known as all leaves being "pure."

In the interactive plot below, the regions are assigned blue and red colors to indicate the predicted class for that region. The shade of the color indicates the predicted probability for that class (darker = higher probability), while yellow regions indicate an equal predicted probability for either class.

### Exercise

**Modify the depth of the tree and see how the paritionning evolves.**

**What can you say about under- and over-fitting of the tree model?**

**How would you choose the best depth?**

In [ ]:
from figures import plot_tree
max_depth = 3
plot_tree(max_depth=max_depth)

## Decision Tree Regression

A decision tree is a simple binary classification tree that is
similar to nearest neighbor classification.  It can be used as follows:

In [ ]:
from figures import make_dataset
x, y = make_dataset()
X = x.reshape(-1, 1)

plt.figure()
plt.xlabel('Feature X')
plt.ylabel('Target y')
plt.scatter(X, y);

In [ ]:
from sklearn.tree import DecisionTreeRegressor

reg = DecisionTreeRegressor(max_depth=None)
reg.fit(X, y)

X_fit = np.linspace(-3, 3, 1000).reshape((-1, 1))
y_fit_1 = reg.predict(X_fit)

plt.figure()
plt.plot(X_fit.ravel(), y_fit_1, color='tab:blue', label="prediction")
plt.plot(X.ravel(), y, 'C7.', label="training data")
plt.legend(loc="best");

A single decision tree allows us to estimate the signal in a non-parametric way,
but clearly has some issues.  In some regions, the model shows high bias and
under-fits the data.
(seen in the long flat lines which don't follow the contours of the data),
while in other regions the model shows high variance and over-fits the data
(reflected in the narrow spikes which are influenced by noise in single points).

### Exercise

**Take the above example and repeat the training/testing by changing depth of the tree.**

**What can you conclude?**

## Bagging classifiers

We saw that by increasing the depth of the tree, we are going to get an over-fitted model. A way to bypass the choice of a specific depth it to combine several trees together.

Let's start by training several trees on slightly different data. The slightly different dataset could be generated by randomly sampling with replacement. In statistics, this called a boostrap sample. We will use the iris dataset to create such ensemble and ensure that we have some data for training and some left out data for testing.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

Before to train several decision trees, we will run a single tree. However, instead to train this tree on `X_train`, we want to train it on a bootstrap sample. You can use the `np.random.choice` function sample with replacement some index. You will need to create a sample_weight vector and pass it to the `fit` method of the `DecisionTreeClassifier`. We provide the `generate_sample_weight` function which will generate the `sample_weight` array.

In [ ]:
def bootstrap_idx(X):
    indices = np.random.choice(
        np.arange(X.shape[0]), size=X.shape[0], replace=True
    )
    return indices

In [ ]:
print(bootstrap_idx(X_train))

In [ ]:
from collections import Counter
print(Counter(bootstrap_idx(X_train)))

In [ ]:
def bootstrap_sample(X, y):
    indices = bootstrap_idx(X)
    return X[indices], y[indices]

In [ ]:
X_train_bootstrap, y_train_bootstrap = bootstrap_sample(X_train, y_train)

In [ ]:
print(f'Classes distribution in the original data: {Counter(y_train)}')
print(f'Classes distribution in the bootstrap: {Counter(y_train_bootstrap)}')

### Exercise: Create a bagging classifier

A bagging classifier will train several decision tree classifiers, each of them on a different bootstrap sample.

* Create several `DecisionTreeClassifier` and store them in a Python list;
* Loop over these trees and `fit` them by generating a bootstrap sample using `bootstrap_sample` function;
* To predict with this ensemble of trees on new data (testing set), you can provide the same set to each tree and call the `predict` method. Aggregate all predictions in a NumPy array;
* Once the predictions available, you need to provide a single prediction: you can retain the class which was the most predicted which is called a majority vote;
* Finally, check the accuracy of your model.

Compute the accuracy score on the training and testing set.

In [ ]:
from sklearn.ensemble import BaggingClassifier

In [ ]:
clf = BaggingClassifier(n_estimators=100)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)

Check the documentation of the `sklearn.ensemble.BaggingClassifier` in scikit-learn. It corresponds to what you implemented up to now.
Apply this classifier on the `digits` dataset and compare the results obtained with a single `sklearn.tree.DecisionTreeClassifier`.

## Random Forests

A very famous classifier is the random forest classifier. It is similar to the bagging classifier. In addition of the bootstrap, the random forest will use a subset of features (selected randomly) to find the best split.

In [ ]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y)

Before to train several decision trees, we will run a single tree. However, instead to train this tree on `X_train`, we want to train it on a bootstrap sample. You can use the `np.random.choice` function sample with replacement some index. You will need to create a sample_weight vector and pass it to the `fit` method of the `DecisionTreeClassifier`. We provide the `generate_sample_weight` function which will generate the `sample_weight` array.

In [ ]:
def generate_sample_weight(X):
    idx_bootstrap = np.random.choice(
        np.arange(X.shape[0]), size=X.shape[0], replace=True
    )
    weight = np.ones(X.shape[0])
    weight *= np.bincount(idx_bootstrap, minlength=X.shape[0])
    return weight

In [ ]:
for _ in range(3):
    print(generate_sample_weight(X_train))

In [ ]:
n_trees = 100
forest = []
for _ in range(n_trees):
    sample_weight = generate_sample_weight(X_train)
    tree = DecisionTreeClassifier(max_features='sqrt')
    forest.append(tree.fit(X_train, y_train, sample_weight=sample_weight))

In [ ]:
y_pred_forest = []
for tree in forest:
    y_pred_forest.append(tree.predict(X_test))

In [ ]:
y_pred_forest = np.array(y_pred_forest).T

In [ ]:
y_pred = []
for sample_pred in y_pred_forest:
    y_pred.append(np.bincount(sample_pred).argmax())

In [ ]:
np.mean(y_pred == y_test)

Use your previous code which was generated several `DecisionTreeClassifier`. Check the list of the option of this classifier and modify one of the parameters such that only the $\sqrt{F}$ features are used for the splitting. $F$ represents the number of features in the dataset.

In [ ]:
from sklearn.datasets import make_blobs
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from figures import plot_2d_separator
from figures import cm2


X, y = make_blobs(centers=[[0, 0], [1, 1]], random_state=61526, n_samples=100)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [ ]:
from sklearn.model_selection import cross_val_score

clf = BaggingClassifier()
cross_val_score(clf, X, y, cv=5)

In scikit-learn, we can directly used the `sklearn.ensemble.RandomForestClassifier`.

In [ ]:
from ipywidgets import interact
from figures import plot_forest
max_depth = 3
interact(plot_forest, max_depth=(1, 10))

## Selecting the Optimal Estimator via Cross-Validation

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.datasets import load_digits
from sklearn.ensemble import RandomForestClassifier

digits = load_digits()
X, y = digits.data, digits.target

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

rf = RandomForestClassifier(n_estimators=200)
parameters = {'max_features':['sqrt', 'log2', 10],
              'max_depth':[5, 7, 9]}

clf_grid = GridSearchCV(rf, parameters, n_jobs=-1)
clf_grid.fit(X_train, y_train)

In [ ]:
clf_grid.score(X_train, y_train)

In [ ]:
clf_grid.score(X_test, y_test)

## Another option: Gradient Boosting

Another Ensemble method that can be useful is *Boosting*: here, rather than
looking at 200 (say) parallel estimators, We construct a chain of 200 estimators
which iteratively refine the results of the previous estimator.
The idea is that by sequentially applying very fast, simple models, we can get a
total model error which is better than any of the individual pieces.

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
clf = GradientBoostingRegressor(n_estimators=100, max_depth=5, learning_rate=.2)
clf.fit(X_train, y_train)

print(clf.score(X_train, y_train))
print(clf.score(X_test, y_test))

<div class="alert alert-success">
    <b>EXERCISE: Cross-validating Gradient Boosting</b>:
     <ul>
      <li>
      Use a grid search to optimize the `learning_rate` and `max_depth` for a Gradient Boosted
Decision tree on the digits data set.
      </li>
    </ul>
</div>

In [ ]:
from sklearn.datasets import load_digits
from sklearn.ensemble import GradientBoostingClassifier

digits = load_digits()
X_digits, y_digits = digits.data, digits.target

# split the dataset, apply grid-search

In [ ]:
# %load solutions/18_gbc_grid.py

## Feature importance

Both RandomForest and GradientBoosting objects expose a `feature_importances_` attribute when fitted. This attribute is one of the most powerful feature of these models. They basically quantify how much each feature contributes to gain in performance in the nodes of the different trees.

In [ ]:
X, y = X_digits[y_digits < 2], y_digits[y_digits < 2]

rf = RandomForestClassifier(n_estimators=300, n_jobs=1)
rf.fit(X, y)
print(rf.feature_importances_)  # one value per feature

In [ ]:
plt.bar(np.arange(rf.feature_importances_.size), rf.feature_importances_)

In [ ]:
plt.figure()
plt.imshow(rf.feature_importances_.reshape(8, 8), cmap=plt.cm.viridis, interpolation='nearest')

### Be careful with feature importances

We will check that we should be careful when interpreting the feature importances when dealing with both numerical and categorical data. Let's take the case of the the titanic dataset.

In [ ]:
import os
import pandas as pd

titanic = pd.read_csv(os.path.join('datasets', 'titanic3.csv'))
titanic.head()

In [ ]:
data = titanic[['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 'embarked']]
labels = titanic.survived.values

In [ ]:
data.head()

We will divide the data into a training and testing set.

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    data, labels, stratify=labels, random_state=42
)

Having categorical and numerical data, we need to have a specific pipeline for the numerical columns and a specific pipeline for the categorical columns.

In [ ]:
categorical_columns = ['pclass', 'sex', 'embarked']

In [ ]:
numerical_columns = ['age', 'sibsp', 'parch', 'fare']

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import OneHotEncoder
from sklearn.impute import SimpleImputer

categorical_pipe = make_pipeline(
    SimpleImputer(strategy='constant', fill_value='missing'),
    OneHotEncoder(handle_unknown='ignore')
)

In [ ]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

numerical_pipe = make_pipeline(
    StandardScaler(), SimpleImputer(strategy='mean')
)

We can use the `sklearn.compose.make_column_transformer` to combine both processing which will be automatically done when calling `fit()` or `transform()`.

In [ ]:
from sklearn.compose import make_column_transformer

preprocessing_pipe = make_column_transformer(
    (categorical_pipe, categorical_columns),
    (numerical_pipe, numerical_columns)
)

Finally, we will use a `RandomForestClassifier` and use the feature importance to check either the numerical or the categorical data are more important.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
final_pipe = make_pipeline(preprocessing_pipe, RandomForestClassifier(random_state=42))

In [ ]:
final_pipe.fit(X_train, y_train).score(X_test, y_test)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(final_pipe.named_steps['randomforestclassifier'].feature_importances_)
plt.xlabel('Feature index')
plt.ylabel('Relative feature importances')

Currently, there is no easy way to `inverse_transform`. However, the last 4 features are the numerical features while the 8 first are the categorical features. Thus, it seems that the numerical features are more important. However, this is due to a bias because of the cardinality which is smaller for categorical data, leading to a reduce amount of split for those features. You can refer to: https://explained.ai/rf-importance/index.html

You can try to:

* use only the categorical features and check the classification performance;
* use only the numerical features and check the classification performance.